# LightGBM v1

### Imports

In [39]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

### Funciones

In [40]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [41]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [42]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [43]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [44]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [45]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [46]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

y_test = y_test.values

x_test = x_test.values

# divido el df 

/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

### Creo LightGBM data containers

In [47]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


In [48]:
#Lo entreno

params = {
    'objective': 'regression',
    'colsample_bytree': 0.9731668400523877,
    'learning_rate': 0.05,
    'min_child_samples': 180,#171 
    'min_child_weight': 1e-05, #1e-05
    'num_leaves': 35,#41 
    'reg_alpha': 10, #10
    'reg_lambda': 1000, #100
    'subsample': 0.9,#0.5575732396028996
    'max_depth':-1, 
    'random_state':314, 
    'silent':True,
    'max_bin': 800,
    'metric':'mae', 
    'n_jobs':None, 
    'n_estimators':100000
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=10000,
                       early_stopping_rounds=500)

[1]	valid_0's l1: 1.55509e+06
Training until validation scores don't improve for 500 rounds
[2]	valid_0's l1: 1.50555e+06
[3]	valid_0's l1: 1.4586e+06
[4]	valid_0's l1: 1.41406e+06
[5]	valid_0's l1: 1.37214e+06
[6]	valid_0's l1: 1.33226e+06
[7]	valid_0's l1: 1.29435e+06
[8]	valid_0's l1: 1.25905e+06
[9]	valid_0's l1: 1.22529e+06
[10]	valid_0's l1: 1.19369e+06
[11]	valid_0's l1: 1.16409e+06
[12]	valid_0's l1: 1.13659e+06
[13]	valid_0's l1: 1.11017e+06
[14]	valid_0's l1: 1.08415e+06
[15]	valid_0's l1: 1.06061e+06
[16]	valid_0's l1: 1.03779e+06
[17]	valid_0's l1: 1.01611e+06
[18]	valid_0's l1: 995145
[19]	valid_0's l1: 976474
[20]	valid_0's l1: 958085
[21]	valid_0's l1: 941133
[22]	valid_0's l1: 924714
[23]	valid_0's l1: 908489
[24]	valid_0's l1: 893707
[25]	valid_0's l1: 879273
[26]	valid_0's l1: 865762
[27]	valid_0's l1: 852679
[28]	valid_0's l1: 840984
[29]	valid_0's l1: 830098
[30]	valid_0's l1: 818957
[31]	valid_0's l1: 808673
[32]	valid_0's l1: 798888
[33]	valid_0's l1: 789503
[34]	

[306]	valid_0's l1: 553265
[307]	valid_0's l1: 553178
[308]	valid_0's l1: 553072
[309]	valid_0's l1: 553003
[310]	valid_0's l1: 552886
[311]	valid_0's l1: 552820
[312]	valid_0's l1: 552717
[313]	valid_0's l1: 552631
[314]	valid_0's l1: 552496
[315]	valid_0's l1: 552399
[316]	valid_0's l1: 552296
[317]	valid_0's l1: 552225
[318]	valid_0's l1: 552133
[319]	valid_0's l1: 552073
[320]	valid_0's l1: 551972
[321]	valid_0's l1: 551893
[322]	valid_0's l1: 551844
[323]	valid_0's l1: 551770
[324]	valid_0's l1: 551713
[325]	valid_0's l1: 551652
[326]	valid_0's l1: 551569
[327]	valid_0's l1: 551473
[328]	valid_0's l1: 551381
[329]	valid_0's l1: 551257
[330]	valid_0's l1: 551171
[331]	valid_0's l1: 551118
[332]	valid_0's l1: 551023
[333]	valid_0's l1: 550965
[334]	valid_0's l1: 550884
[335]	valid_0's l1: 550813
[336]	valid_0's l1: 550748
[337]	valid_0's l1: 550636
[338]	valid_0's l1: 550577
[339]	valid_0's l1: 550499
[340]	valid_0's l1: 550446
[341]	valid_0's l1: 550302
[342]	valid_0's l1: 550245
[

[610]	valid_0's l1: 537318
[611]	valid_0's l1: 537283
[612]	valid_0's l1: 537240
[613]	valid_0's l1: 537212
[614]	valid_0's l1: 537185
[615]	valid_0's l1: 537165
[616]	valid_0's l1: 537147
[617]	valid_0's l1: 537126
[618]	valid_0's l1: 537118
[619]	valid_0's l1: 537064
[620]	valid_0's l1: 537050
[621]	valid_0's l1: 537021
[622]	valid_0's l1: 536989
[623]	valid_0's l1: 536957
[624]	valid_0's l1: 536937
[625]	valid_0's l1: 536924
[626]	valid_0's l1: 536915
[627]	valid_0's l1: 536883
[628]	valid_0's l1: 536847
[629]	valid_0's l1: 536831
[630]	valid_0's l1: 536794
[631]	valid_0's l1: 536772
[632]	valid_0's l1: 536743
[633]	valid_0's l1: 536696
[634]	valid_0's l1: 536659
[635]	valid_0's l1: 536591
[636]	valid_0's l1: 536555
[637]	valid_0's l1: 536537
[638]	valid_0's l1: 536517
[639]	valid_0's l1: 536459
[640]	valid_0's l1: 536429
[641]	valid_0's l1: 536403
[642]	valid_0's l1: 536395
[643]	valid_0's l1: 536382
[644]	valid_0's l1: 536331
[645]	valid_0's l1: 536305
[646]	valid_0's l1: 536291
[

[917]	valid_0's l1: 530737
[918]	valid_0's l1: 530735
[919]	valid_0's l1: 530717
[920]	valid_0's l1: 530698
[921]	valid_0's l1: 530664
[922]	valid_0's l1: 530653
[923]	valid_0's l1: 530637
[924]	valid_0's l1: 530628
[925]	valid_0's l1: 530615
[926]	valid_0's l1: 530607
[927]	valid_0's l1: 530603
[928]	valid_0's l1: 530573
[929]	valid_0's l1: 530560
[930]	valid_0's l1: 530528
[931]	valid_0's l1: 530534
[932]	valid_0's l1: 530521
[933]	valid_0's l1: 530517
[934]	valid_0's l1: 530507
[935]	valid_0's l1: 530503
[936]	valid_0's l1: 530485
[937]	valid_0's l1: 530470
[938]	valid_0's l1: 530474
[939]	valid_0's l1: 530449
[940]	valid_0's l1: 530442
[941]	valid_0's l1: 530423
[942]	valid_0's l1: 530409
[943]	valid_0's l1: 530377
[944]	valid_0's l1: 530355
[945]	valid_0's l1: 530329
[946]	valid_0's l1: 530312
[947]	valid_0's l1: 530303
[948]	valid_0's l1: 530307
[949]	valid_0's l1: 530294
[950]	valid_0's l1: 530267
[951]	valid_0's l1: 530257
[952]	valid_0's l1: 530241
[953]	valid_0's l1: 530232
[

[1218]	valid_0's l1: 526654
[1219]	valid_0's l1: 526645
[1220]	valid_0's l1: 526624
[1221]	valid_0's l1: 526617
[1222]	valid_0's l1: 526601
[1223]	valid_0's l1: 526592
[1224]	valid_0's l1: 526587
[1225]	valid_0's l1: 526583
[1226]	valid_0's l1: 526569
[1227]	valid_0's l1: 526567
[1228]	valid_0's l1: 526559
[1229]	valid_0's l1: 526525
[1230]	valid_0's l1: 526521
[1231]	valid_0's l1: 526512
[1232]	valid_0's l1: 526495
[1233]	valid_0's l1: 526480
[1234]	valid_0's l1: 526474
[1235]	valid_0's l1: 526458
[1236]	valid_0's l1: 526446
[1237]	valid_0's l1: 526437
[1238]	valid_0's l1: 526434
[1239]	valid_0's l1: 526429
[1240]	valid_0's l1: 526402
[1241]	valid_0's l1: 526395
[1242]	valid_0's l1: 526384
[1243]	valid_0's l1: 526378
[1244]	valid_0's l1: 526374
[1245]	valid_0's l1: 526357
[1246]	valid_0's l1: 526343
[1247]	valid_0's l1: 526335
[1248]	valid_0's l1: 526319
[1249]	valid_0's l1: 526308
[1250]	valid_0's l1: 526304
[1251]	valid_0's l1: 526293
[1252]	valid_0's l1: 526283
[1253]	valid_0's l1:

...

[18372]	valid_0's l1: 502402
[18373]	valid_0's l1: 502403
[18374]	valid_0's l1: 502404
[18375]	valid_0's l1: 502405
[18376]	valid_0's l1: 502405
[18377]	valid_0's l1: 502404
[18378]	valid_0's l1: 502404
[18379]	valid_0's l1: 502404
[18380]	valid_0's l1: 502405
[18381]	valid_0's l1: 502405
[18382]	valid_0's l1: 502406
[18383]	valid_0's l1: 502407
[18384]	valid_0's l1: 502407
[18385]	valid_0's l1: 502404
[18386]	valid_0's l1: 502403
[18387]	valid_0's l1: 502403
[18388]	valid_0's l1: 502404
[18389]	valid_0's l1: 502403
[18390]	valid_0's l1: 502398
[18391]	valid_0's l1: 502399
[18392]	valid_0's l1: 502400
[18393]	valid_0's l1: 502400
[18394]	valid_0's l1: 502400
[18395]	valid_0's l1: 502400
[18396]	valid_0's l1: 502399
[18397]	valid_0's l1: 502401
[18398]	valid_0's l1: 502401
[18399]	valid_0's l1: 502400
[18400]	valid_0's l1: 502404
[18401]	valid_0's l1: 502406
[18402]	valid_0's l1: 502408
[18403]	valid_0's l1: 502409
[18404]	valid_0's l1: 502408
[18405]	valid_0's l1: 502407
[18406]	valid_

[18660]	valid_0's l1: 502331
[18661]	valid_0's l1: 502333
[18662]	valid_0's l1: 502331
[18663]	valid_0's l1: 502331
[18664]	valid_0's l1: 502331
[18665]	valid_0's l1: 502328
[18666]	valid_0's l1: 502327
[18667]	valid_0's l1: 502329
[18668]	valid_0's l1: 502328
[18669]	valid_0's l1: 502328
[18670]	valid_0's l1: 502328
[18671]	valid_0's l1: 502328
[18672]	valid_0's l1: 502327
[18673]	valid_0's l1: 502327
[18674]	valid_0's l1: 502328
[18675]	valid_0's l1: 502328
[18676]	valid_0's l1: 502328
[18677]	valid_0's l1: 502329
[18678]	valid_0's l1: 502325
[18679]	valid_0's l1: 502326
[18680]	valid_0's l1: 502326
[18681]	valid_0's l1: 502327
[18682]	valid_0's l1: 502329
[18683]	valid_0's l1: 502330
[18684]	valid_0's l1: 502329
[18685]	valid_0's l1: 502330
[18686]	valid_0's l1: 502330
[18687]	valid_0's l1: 502331
[18688]	valid_0's l1: 502334
[18689]	valid_0's l1: 502335
[18690]	valid_0's l1: 502336
[18691]	valid_0's l1: 502336
[18692]	valid_0's l1: 502339
[18693]	valid_0's l1: 502338
[18694]	valid_

[18949]	valid_0's l1: 502341
[18950]	valid_0's l1: 502343
[18951]	valid_0's l1: 502343
[18952]	valid_0's l1: 502343
[18953]	valid_0's l1: 502341
[18954]	valid_0's l1: 502343
[18955]	valid_0's l1: 502343
[18956]	valid_0's l1: 502343
[18957]	valid_0's l1: 502343
[18958]	valid_0's l1: 502343
[18959]	valid_0's l1: 502345
[18960]	valid_0's l1: 502345
[18961]	valid_0's l1: 502345
[18962]	valid_0's l1: 502343
[18963]	valid_0's l1: 502343
[18964]	valid_0's l1: 502342
[18965]	valid_0's l1: 502343
[18966]	valid_0's l1: 502345
[18967]	valid_0's l1: 502347
[18968]	valid_0's l1: 502349
[18969]	valid_0's l1: 502349
[18970]	valid_0's l1: 502349
[18971]	valid_0's l1: 502348
[18972]	valid_0's l1: 502345
[18973]	valid_0's l1: 502344
[18974]	valid_0's l1: 502345
[18975]	valid_0's l1: 502345
[18976]	valid_0's l1: 502344
[18977]	valid_0's l1: 502342
[18978]	valid_0's l1: 502341
[18979]	valid_0's l1: 502343
[18980]	valid_0's l1: 502342
[18981]	valid_0's l1: 502342
[18982]	valid_0's l1: 502342
[18983]	valid_

[19237]	valid_0's l1: 502315
[19238]	valid_0's l1: 502314
[19239]	valid_0's l1: 502314
[19240]	valid_0's l1: 502311
[19241]	valid_0's l1: 502312
[19242]	valid_0's l1: 502313
[19243]	valid_0's l1: 502313
[19244]	valid_0's l1: 502314
[19245]	valid_0's l1: 502317
[19246]	valid_0's l1: 502317
[19247]	valid_0's l1: 502318
[19248]	valid_0's l1: 502318
[19249]	valid_0's l1: 502320
[19250]	valid_0's l1: 502318
[19251]	valid_0's l1: 502317
[19252]	valid_0's l1: 502319
[19253]	valid_0's l1: 502320
[19254]	valid_0's l1: 502319
[19255]	valid_0's l1: 502321
[19256]	valid_0's l1: 502321
[19257]	valid_0's l1: 502320
[19258]	valid_0's l1: 502319
[19259]	valid_0's l1: 502320
[19260]	valid_0's l1: 502320
[19261]	valid_0's l1: 502321
[19262]	valid_0's l1: 502322
[19263]	valid_0's l1: 502320
[19264]	valid_0's l1: 502321
[19265]	valid_0's l1: 502321
[19266]	valid_0's l1: 502321
[19267]	valid_0's l1: 502322
[19268]	valid_0's l1: 502323
[19269]	valid_0's l1: 502323
[19270]	valid_0's l1: 502324
[19271]	valid_

[19521]	valid_0's l1: 502359
[19522]	valid_0's l1: 502362
[19523]	valid_0's l1: 502364
[19524]	valid_0's l1: 502364
[19525]	valid_0's l1: 502362
[19526]	valid_0's l1: 502363
[19527]	valid_0's l1: 502362
[19528]	valid_0's l1: 502359
[19529]	valid_0's l1: 502360
[19530]	valid_0's l1: 502363
[19531]	valid_0's l1: 502364
[19532]	valid_0's l1: 502364
[19533]	valid_0's l1: 502367
[19534]	valid_0's l1: 502370
[19535]	valid_0's l1: 502373
[19536]	valid_0's l1: 502374
[19537]	valid_0's l1: 502375
[19538]	valid_0's l1: 502375
[19539]	valid_0's l1: 502374
[19540]	valid_0's l1: 502373
[19541]	valid_0's l1: 502372
[19542]	valid_0's l1: 502372
[19543]	valid_0's l1: 502372
[19544]	valid_0's l1: 502372
[19545]	valid_0's l1: 502372
[19546]	valid_0's l1: 502371
[19547]	valid_0's l1: 502372
[19548]	valid_0's l1: 502373
[19549]	valid_0's l1: 502373
[19550]	valid_0's l1: 502373
[19551]	valid_0's l1: 502373
[19552]	valid_0's l1: 502373
[19553]	valid_0's l1: 502374
[19554]	valid_0's l1: 502374
[19555]	valid_

In [129]:
prediccion = model.predict(x_test)

KeyboardInterrupt: 

In [ ]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

# gridsearch

In [45]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'regression',
          'nthread': None, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# creo los parametros para grid  
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [1,2],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [501], 
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# pongo los parametros 
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression',
          n_jobs = None, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

#default:
mdl.get_params().keys()

# Armo grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=None)
# Enteno
grid.fit(x, y_train)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_bin=512, max_depth=-1,
                                     min_child_samples=5, min_child_weight=1,
                                     min_split_gain=0.5, n_estimators=100,
                                     n_jobs=None, num_leaves=31,
                                     objective='binary', random_state=None,
                                     reg_alpha=0...
             iid='warn', n_jobs=None,
             param_grid={'boosting_type': ['gbdt'],
                         'colsample_bytree': [0.65, 0.66],
                         'learning_rate': [0.005], 'n_estimators': [1, 2],
                         'num_leaves': [6, 8, 12, 16],
                         'objective': ['r

In [48]:
grid.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.65,
 'learning_rate': 0.005,
 'n_estimators': 2,
 'num_leaves': 16,
 'objective': 'regression',
 'random_state': 501,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.7}

In [46]:
prediccion = grid.predict(x_test)

In [47]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

1600921.908940707


In [94]:
561929.2799609166 100

SyntaxError: invalid syntax (<ipython-input-94-41a49510c087>, line 1)

# Random Search

In [32]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

sp_uniform(loc=0.2, scale=0.8)

In [42]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}
print (param_test)

{'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f54767ac278>, 'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f54767aaba8>, 'min_child_weight': [1e-05, 0.001, 0.01, 0.1, 1, 10.0, 100.0, 1000.0, 10000.0], 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f54763aef98>, 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f54763aed68>, 'reg_alpha': [0, 0.1, 1, 2, 5, 7, 10, 50, 100], 'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50, 100]}


In [43]:
#Numero de ptos a probar 
n_HP_points_to_test = 15

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='mae', n_jobs=None, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)


In [ ]:
gs.fit(x, y_train)
print('Best score with params: {} '.format( gs.best_params_))

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
prediccion = gs.predict(x_test)

In [ ]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

In [ ]:
509
